# EDA 1st round

這是拿到初步整理資料後進行的前期探索，因為資料來源多種，有VD也有ETag，兩者的精度、粒度不同  
在交通領域上時間序列會跟路段型態有相當大的關係，這邊為了先簡化題目的範圍，會聚焦在國道五號的資料上  
EDA的過程是會來來回回的，敲定範圍後可能需要在對資料進行額外的加工後才能更進一步探索  
本notebook先做第一次的EDA，看的資料會直接以原始檔做為參考  

後續會以Etag的資料為主，因此VD的簡易帶過，讓大家能清楚結構跟差異

# Lib

In [1]:
import pandas as pd 
import numpy as np
import json
import sqlite3

部分的資料已經先儲存在DB中了，我們可以透過以下方式進行簡單的query  

In [ ]:
db_path = "../data/hwdb.db"

# VD

## links and geo info

這邊的資料是前面儲存在VD Static的內容，主要記錄的是道路幾何型態資訊  
正常狀況下每天會是一樣的，除非該VD故障或者是被移除，通常這樣的資訊在較為長期的觀測中才會被放大  
如果我們要做的是預測路段的話，只要掌握link and node資訊即可
不太需要每天去鎖定VD的data  

In [ ]:
VD的資料我們這邊做個簡易的探勘，帶大家看看在國道五號上相關的資料複雜程度

In [ ]:
這邊先簡單抽取

In [13]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM VD_STATIC
    WHERE RoadName == "國道5號"
    ''',
    con)
con.close()

In [16]:
df.head(3)

,UpdateTime,UpdateInterval,AuthorityCode,VDID,SubAuthorityCode,BiDirectional,LinkID,Bearing,RoadDirection,Lane,...,LocationType,DetectionType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,Start,End,LocationMile
0,2023-11-01 00:00:00+08:00,86400,NFB,VD-N5-S-0.191-M-LOOP,NFB-PL,0,0000500000000A,E,S,2,...,5,1,121.62496,25.034428,000050,國道5號,0,南港系統交流道,石碇交流道,0K+191
1,2023-11-01 00:00:00+08:00,86400,NFB,VD-N5-S-12.945-M-LOOP,NFB-PL,0,0000500001200F,SE,S,2,...,5,1,121.69732,24.949984,000050,國道5號,0,石碇交流道,坪林交控交流道,12K+945
2,2023-11-01 00:00:00+08:00,86400,NFB,VD-N5-S-27.748-M-LOOP,NFB-PL,0,0000500002700G,S,S,2,...,5,1,121.79097,24.851341,000050,國道5號,0,坪林交控交流道,頭城交流道,27K+748


In [20]:
print(f'VDID 在國道五號上有 {df.VDID.nunique()} 個')
display(df.groupby(['RoadDirection', 'Lane', 'LocationType']).agg({'VDID': 'count'}))

VDID 在國道五號上有 140 個


VDID
RoadDirection Lane LocationType      
N             1    5               16
              2    5               52
              3    5                8
S             1    5               13
              2    5               46
              3    5                5

根據指定維度可以計算VD的數量，南北向數量不太一樣，各車道上的分布也不同  
從這邊可以先簡易的概括如果採用VD我們要研究的位置就相當複雜了，後面還涉及車種速率等指標的計算  
單一個方向就有76~64個偵測器，往下需要再通過link到link進行劃分，對於追求複雜的路網呈現  
或是要求更高的分析專案而言，可以此作為入門方向，只是後續要處理的資料工程難度更高  
本專案就不採納VD的資料作為研究方向，僅提供一個方向參考  

## traffic vol and status

這邊看的就會是VD Dynamic中存放的內容了  
基本上會有特定VD的資訊，放置的車道、設備狀態、偵測速率、佔有率等等  
因為資料量相當大，我們根據前一節的其中一個VDID切入大概看一下資料就好  

In [22]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM VD_DYNAMIC
    WHERE VDID == "VD-N5-S-0.191-M-LOOP"
    ''',
    con)
con.close()

In [24]:
display(df.shape)
df.head(2)

(8730, 14)

,UpdateTime,UpdateInterval,AuthorityCode,VDID,LinkID,LaneID,LaneType,Speed,Occupancy,VehicleType,Volume,Speed2,Status,DataCollectTime
0,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,0,2,-99,-99,S,-99,-99,1,2023-10-31 23:58:00+08:00
1,2023-11-01 00:00:00+08:00,60,NFB,VD-N5-S-0.191-M-LOOP,0000500000000A,0,2,-99,-99,L,-99,-99,1,2023-10-31 23:58:00+08:00


目前只有塞一天的資料在裡面但不加條件的話數量是很驚人的，透過抽取特定VDID我們可以檢視一下  
其中就出現了好幾個資料異常  
這種異常說明了，如果要好好利用VD資料我們還需要針對這種異常進行缺補  
否則正常下我們可以直接就車種去計算加權平均的車速跟流量了  

In [26]:
df.VehicleType.value_counts()

VehicleType
S    2910
L    2910
T    2910
Name: count, dtype: int64

# ETag

## Road and sensor position

In [5]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_STATIC
    --WHERE VDID == "VD-N5-S-0.191-M-LOOP"
    ''',
    con)
con.close()

In [6]:
df.groupby(['RoadName', 'RoadDirection']).agg({'ETagGantryID': 'count'})

ETagGantryID
RoadName     RoadDirection              
國道1號         N                        75
             S                        75
國道1號五股楊梅高架道路 N                         4
             S                         4
國道1號汐止五股高架道路 N                         4
             S                         3
國道3甲         N                         2
             S                         2
國道3號         N                        78
             S                        78
國道5號         N                         7
             S                         7

本次會聚焦在國道5號，因為從路線的幾何型態來看是相對簡單，不需要處理太多分支跟上下游分岔的狀況

與VD的幾何型態資訊一樣，在時間較長的範圍下，sensor的位置可能會變化  
道路的長度概念上不會改變，但從資料面而言可能會因為路線繪製的調整而有所變化  
LinkVersion如果把資料時間範圍拉長，就有機會觀測到版本變化  

In [ ]:
南北向的起始點有稍微不太一樣

In [21]:
# 這邊我們聚焦國五的北向
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    """
    SELECT * 
    FROM ETAG_STATIC
    WHERE RoadDirection == 'N'
    AND RoadID == '000050'
    """,
    con
)
con.close()

In [22]:
df

,UpdateTime,UpdateInterval,AuthorityCode,LinkVersion,ETagGantryID,LinkID,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,Start,End,LocationMile
0,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0001N,0000500100000A,4,121.62472,25.035036,000050,國道5號,0,N,石碇,南港系統,0K+100
1,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0309N,0000500103100G,4,121.78635,24.823656,000050,國道5號,0,N,宜蘭(四城、大福),頭城,30K+900
2,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0528N,0000500105300G,4,121.80696,24.632729,000050,國道5號,0,N,蘇澳,羅東,52K+800
3,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05FR143N,0000501105010G,4,121.782364,24.733961,000050,國道5號,0,N,宜蘭(壯圍),宜蘭(四城、大福),14K+300
4,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0055N,0000500100500F,4,121.652084,24.996504,000050,國道5號,0,N,坪林交控中心專用道,石碇,5K+500
5,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0287N,0000500102820G,4,121.789185,24.842714,000050,國道5號,0,N,頭城,坪林交控中心專用道,28K+700
6,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0438N,0000500104400G,4,121.789474,24.711027,000050,國道5號,0,N,羅東,宜蘭(壯圍),43K+800


理論上國五的etc門架由北至南順序應該會是座路在這些地方中間  
* 南港系統
* 石碇
* 坪林交控中心專用道
* 頭城
* 宜蘭(四城、大福) 北入南出
* 宜蘭(壯圍) 南入北出
* 羅東
* 蘇澳

所以由北至南的順序應該會是  
* 05F0001N
* 05F0055N
* 05F0287N
* 05F0309N
* 05FR143N
* 05F0438N
* 05F0528N

In [23]:
df['SerialMile'] = df['LocationMile'].apply(lambda x: 
                                            int(x.split('+')[0].replace('K',''))*1000 
                                            + int(x.split('+')[1]))

In [25]:
# 手動只要排好下面的順序，可以用list + dict產生兌換用的功能
loc_list = ['蘇澳', '羅東', '宜蘭(壯圍)', '宜蘭(四城、大福)', '頭城', '坪林交控中心專用道', '石碇', '南港系統']
order_list = [i for i in range(len(loc_list))]
nd_order_dict = dict(zip(loc_list, order_list))


# 加上排序用cols
df['loc_order']=df['Start'].apply(lambda x: nd_order_dict[x])
df.sort_values(by=['loc_order'], inplace=True)

serial mile不太能作為順序參考，因為從地理上來說順序與位置未必成正向關係  

In [26]:
df

,UpdateTime,UpdateInterval,AuthorityCode,LinkVersion,ETagGantryID,LinkID,LocationType,PositionLon,PositionLat,RoadID,RoadName,RoadClass,RoadDirection,Start,End,LocationMile,SerialMile,loc_order
2,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0528N,0000500105300G,4,121.80696,24.632729,000050,國道5號,0,N,蘇澳,羅東,52K+800,52800,0
6,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0438N,0000500104400G,4,121.789474,24.711027,000050,國道5號,0,N,羅東,宜蘭(壯圍),43K+800,43800,1
3,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05FR143N,0000501105010G,4,121.782364,24.733961,000050,國道5號,0,N,宜蘭(壯圍),宜蘭(四城、大福),14K+300,14300,2
1,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0309N,0000500103100G,4,121.78635,24.823656,000050,國道5號,0,N,宜蘭(四城、大福),頭城,30K+900,30900,3
5,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0287N,0000500102820G,4,121.789185,24.842714,000050,國道5號,0,N,頭城,坪林交控中心專用道,28K+700,28700,4
4,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0055N,0000500100500F,4,121.652084,24.996504,000050,國道5號,0,N,坪林交控中心專用道,石碇,5K+500,5500,5
0,2023-11-01T00:00:00+08:00,86400,NFB,23.05.1,05F0001N,0000500100000A,4,121.62472,25.035036,000050,國道5號,0,N,石碇,南港系統,0K+100,100,6


In [9]:
target_etag_gantry = df.query("LinkVersion == '23.05.1' and RoadName == '國道5號'").ETagGantryID.unique()

In [ ]:
這邊要補上門架的前後順序

## ETag pair info

從前面先取得門架的資訊，方便後續進行限縮

In [10]:
target_etag_gantry

array(['05F0000S', '05F0001N', '05FR113S', '05F0309N', '05F0309S',
       '05F0528N', '05FR143N', '05F0055S', '05F0055N', '05F0287S',
       '05F0287N', '05F0494S', '05F0438N', '05F0439S'], dtype=object)

In [27]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_PAIR
    WHERE ETagPairID in ("05F0001N-03F0150N", 
                         "05F0001N-03F0201S",
                         "05F0055N-05F0001N",
                         "05F0287N-05F0055N",
                         "05F0309N-05F0287N",
                         "05F0438N-05F0309N",
                         "05F0438N-05FR143N",
                         "05F0528N-05F0438N")
    ''',
    con)
con.close()

In [28]:
df.head()

,UpdateTime,UpdateInterval,AuthorityCode,ETagPairID,StartETagGantryID,EndETagGantryID,Description,Distance,StartLinkID,EndLinkID,Geometry
0,2023-11-01T00:00:00+08:00,86400,NFB,05F0438N-05F0309N,05F0438N,05F0309N,羅東-宜蘭(四城、大福),12.8,0000500104300G,0000500103000G,"LINESTRING(121.78873 24.71428,121.78849 24.715..."
1,2023-11-01T00:00:00+08:00,86400,NFB,05F0528N-05F0438N,05F0528N,05F0438N,蘇澳-羅東,8.76,0000500105200G,0000500104300G,"LINESTRING(121.80576 24.63708,121.80574 24.637..."
2,2023-11-01T00:00:00+08:00,86400,NFB,05F0055N-05F0001N,05F0055N,05F0001N,坪林交控中心專用道-石碇,5.28,0000500100500F,0000500100000A,"LINESTRING(121.65206 24.99663,121.65205 24.996..."
3,2023-11-01T00:00:00+08:00,86400,NFB,05F0309N-05F0287N,05F0309N,05F0287N,宜蘭(四城、大福)-頭城,2.13,0000500103090G,0000500102820G,"LINESTRING(121.78757 24.82461,121.78762 24.824..."
4,2023-11-01T00:00:00+08:00,86400,NFB,05F0438N-05FR143N,05F0438N,05FR143N,羅東-宜蘭(壯圍),0,0000500104300G,0000500101400F,"LINESTRING(121.78873 24.71428,121.78849 24.715..."


In [15]:
df.keys()

Index(['UpdateTime', 'UpdateInterval', 'AuthorityCode', 'ETagPairID',
       'StartETagGantryID', 'EndETagGantryID', 'Description', 'Distance',
       'StartLinkID', 'EndLinkID', 'Geometry'],
      dtype='object')

In [17]:
df[df.StartETagGantryID.isin(target_etag_gantry)\
   & df.EndETagGantryID.isin(target_etag_gantry)]\
[['Description', 'Distance', 'EndETagGantryID', 'StartETagGantryID']]

,Description,Distance,EndETagGantryID,StartETagGantryID
15,羅東-宜蘭(四城、大福),12.8,05F0309N,05F0438N
51,蘇澳-羅東,8.76,05F0438N,05F0528N
58,坪林交控中心專用道-石碇,5.28,05F0001N,05F0055N
84,宜蘭(壯圍)-羅東,5.28,05F0494S,05F0439S
138,坪林交控中心專用道-頭城,2.16,05F0309S,05F0287S
140,宜蘭(四城、大福)-頭城,2.13,05F0287N,05F0309N
153,羅東-宜蘭(壯圍),0,05FR143N,05F0438N
158,頭城-坪林交控中心專用道,22.95,05F0055N,05F0287N
187,宜蘭(四城、大福)-宜蘭(壯圍),3.11,05F0439S,05FR113S
233,頭城-宜蘭(壯圍),12.78,05F0439S,05F0309S


In [ ]:
上述的資訊中包含了靜態的起訖對距離作為參考

## ETag pair dynamic info

這邊來看一下動態資訊中的變化吧  
把南北兩個方向都拉進來參考  

In [29]:
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_PAIR_LIVE
    WHERE ETagPairID in ('05F0438N-05F0309N', '05F0528N-05F0438N', '05F0055N-05F0001N',
                         '05F0439S-05F0494S', '05F0287S-05F0309S', '05F0309N-05F0287N',
                         '05F0438N-05FR143N', '05F0287N-05F0055N', '05FR113S-05F0439S',
                         '05F0309S-05F0439S', '05F0000S-05F0055S', '05F0055S-05F0287S')
    ''',
    con)
con.close()

In [30]:
df

,UpdateTime,UpdateInterval,AuthorityCode,ETagPairID,StartETagStatus,EndETagStatus,VehicleType,TravelTime,StandardDeviation,SpaceMeanSpeed,VehicleCount,StartTime,EndTime,DataCollectTime
0,2023-11-01 00:00:00+08:00,300,NFB,05F0528N-05F0438N,0,0,31,374,0,86,1,2023-10-31 23:30:00+08:00,2023-10-31 23:35:00+08:00,2023-10-31 23:35:00+08:00
1,2023-11-01 00:00:00+08:00,300,NFB,05F0528N-05F0438N,0,0,32,332,0,95,2,2023-10-31 23:30:00+08:00,2023-10-31 23:35:00+08:00,2023-10-31 23:35:00+08:00
2,2023-11-01 00:00:00+08:00,300,NFB,05F0528N-05F0438N,0,0,41,0,0,0,0,2023-10-31 23:30:00+08:00,2023-10-31 23:35:00+08:00,2023-10-31 23:35:00+08:00
3,2023-11-01 00:00:00+08:00,300,NFB,05F0528N-05F0438N,0,0,42,408,0,79,2,2023-10-31 23:30:00+08:00,2023-10-31 23:35:00+08:00,2023-10-31 23:35:00+08:00
4,2023-11-01 00:00:00+08:00,300,NFB,05F0528N-05F0438N,0,0,5,332,0,97,1,2023-10-31 23:30:00+08:00,2023-10-31 23:35:00+08:00,2023-10-31 23:35:00+08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,2023-11-01 23:55:00+08:00,300,NFB,05F0309S-05F0439S,0,0,31,492,0,93,17,2023-11-01 23:30:00+08:00,2023-11-01 23:35:00+08:00,2023-11-01 23:35:00+08:00
17276,2023-11-01 23:55:00+08:00,300,NFB,05F0309S-05F0439S,0,0,32,477,0,97,3,2023-11-01 23:30:00+08:00,2023-11-01 23:35:00+08:00,2023-11-01 23:35:00+08:00
17277,2023-11-01 23:55:00+08:00,300,NFB,05F0309S-05F0439S,0,0,41,531,0,86,1,2023-11-01 23:30:00+08:00,2023-11-01 23:35:00+08:00,2023-11-01 23:35:00+08:00
17278,2023-11-01 23:55:00+08:00,300,NFB,05F0309S-05F0439S,0,0,42,0,0,0,0,2023-11-01 23:30:00+08:00,2023-11-01 23:35:00+08:00,2023-11-01 23:35:00+08:00


請注意，上述的是原始資料，每五分鐘為單位計算  
包含了車種、旅行時間、空間平均速率、車輛數等統計  
後續我們可以使用其他一樣是ETag組成的資料進行分析  
這邊指大概帶過db擷取完畢的資料可以如何叫出跟解讀  

In [ ]:
在進入本專案核心的EDA之前，因為硬體需求只能先將我需要觀察的資料拉出來另存成csv  
你想要用parquet也可以  

# Limit the range of target data

In [ ]:
以下大概註記一下DB這段我做了哪些事情  
儲存了多少跟擷取了多少資料出來  
好真正開始第二輪的EDA  

In [ ]:
鎖定國道五號、往北方向的資料為主  
注意，我放在db中的資料量在預設是相當大的，因此實際拉出來應該會跟前面看到的不太一樣  

In [ ]:
# Etag location
con = sqlite3.connect("../data/hwdb.db")
df = pd.read_sql_query(
    '''
    SELECT * 
    FROM ETAG_STATIC
    WHERE RoadDirection == 'N'
    AND RoadID == '000050'
    ''',
    con)
con.close()

Etag_5n_loc = df.drop(columns=['UpdateInterval', 'AuthorityCode', 'RoadClass', 'LocationType']) \
                .drop_duplicates() \
                .copy()

Etag_5n_loc.to_csv('../data/cleaned/Etag_5n_loc.csv', index=False)

In [ ]:
原始的M04A相當大，為了讓sqlite能負荷是以每個月作為一張資料表處理  
以下提供撈取的方法  

In [24]:
# M04A

table_time = ['202301', '202302' , '202303', '202304', '202305',
              '202306', '202307', '202308', '202309', '202310',
              '202311', '202312', '202401', '202402', '202403']

hw5_m04a = pd.DataFrame()

for year_month in table_time:
    db_table = 'ETAG_M04A_' + year_month
    con = sqlite3.connect("../data/hwdb.db")
    df = pd.read_sql_query(
        f'''
        SELECT * 
        FROM {db_table}
        WHERE GantryFrom in ('05F0438N', '05F0309N', '05F0528N', '05F0055N', '05F0001N',
        '05F0287N', '05FR143N', '03F0150N', '03F0201S')
        ''', con)
    con.close()
    hw5_m04a = pd.concat([hw5_m04a, df], axis=0)
    print(f'finish loading table {db_table}')

In [ ]:
hw5_m04a.to_csv('../data/cleaned/hw5_m04a.csv', index=False)

In [ ]:
# section
section = pd.read_csv('./out/link_v2.csv')
section.query('RoadID == "000050" and RoadDirection == "N"').to_csv('hw5_sectionlink.csv',index=False)

In [ ]:
# congestion_table

In [ ]:
# calendar_event

In [ ]:
# road_build_event

In [ ]:
# traffic_accident_data